In [1]:
%matplotlib inline

import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
from cselect import color as cs
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../samples')
from mvpose.data import epfl_campus
from time import time

root = Settings['data_root']
root = join(root, 'pak')
tmp = Settings['tmp']

import mvpose.data.kth_football2 as kth
from mvpose import pose
from mvpose.settings import get_settings
from paf_loader import Loader
from mvpose.evaluation import pcp
from mvpose.data import umpm

root = join(Settings['data_root'], 'pak')
user = Settings['UMPM']['username']
pwd = Settings['UMPM']['password']
tmp = Settings['tmp']
tmp = '/tmp'

params = get_settings()
loader = Loader(with_gpu=True)

X, Y, Calib = umpm.get(root, 'p2_free_1', user, pwd)

/home/tanke/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


UMPM


In [2]:
def generate_pcp_score(frame):
    alpha = 0.2
    Im = np.array([X[0][frame], X[1][frame], X[2][frame], X[3][frame]])
    heatmaps, pafs = loader.load_confidence_map_and_paf('umpm', 
                                                            Im, 
                                                            frame, 
                                                            dir=tmp)
    
    detections = pose.estimate(
        Calib, heatmaps, pafs, settings=params, debug=False)
    Humans = kth.transform3d_from_mscoco(detections)
    
    L_Arms = []
    U_Arms = []
    L_Legs = []
    U_Legs = []
    GTIDs = []
    a = Y[frame,0:14,0:3]
    b = Y[frame,14:,0:3]
    Y_ = kth.transform3d_from_umpm([a, b])
    
    for gtid, gt in enumerate(Y_):
        if gt is None:
            continue
        
        larms = 0
        uarms = 0
        llegs = 0
        ulegs = 0
        avg = 0
        for d in Humans:
            r = pcp.evaluate(gt, d, alpha)
            larms_ = r.lower_arms
            uarms_ = r.upper_arms
            ulegs_ = r.upper_legs
            llegs_ = r.lower_legs
            avg_ = (larms_ + uarms_ + ulegs_ + llegs_) / 4
            if avg_ > avg:
                avg = avg_
                larms = larms_
                uarms = uarms_
                llegs = llegs_
                ulegs = ulegs_
        
        L_Arms.append(larms)
        U_Arms.append(uarms)
        L_Legs.append(llegs)
        U_Legs.append(ulegs)
        GTIDs.append(gtid)
    
    return L_Arms, U_Arms, L_Legs, U_Legs, GTIDs, detections


Detections = []
PER_GTID = {}
n = 2437
#n = 20
for frame in range(0, n, 5):
    _start = time()
    L_Arms, U_Arms, L_Legs, U_Legs, GTIDs, detections =\
        generate_pcp_score(frame)
    Detections.append(detections)
    _end = time()
    print('frame ' + str(frame) + ', elapsed', _end - _start)
    
    if len(L_Arms) > 0:
        for gtid, larms, uarms, llegs, ulegs in zip(
            GTIDs, L_Arms, U_Arms, L_Legs, U_Legs
        ):
            if not gtid in PER_GTID:
                PER_GTID[gtid] = {
                    'larms': [],
                    'uarms': [],
                    'llegs': [],
                    'ulegs': [],
                    'frame': []
                }
            PER_GTID[gtid]['larms'].append(larms)
            PER_GTID[gtid]['uarms'].append(uarms)
            PER_GTID[gtid]['llegs'].append(llegs)
            PER_GTID[gtid]['ulegs'].append(ulegs)
            PER_GTID[gtid]['frame'].append(frame)

frame 0, elapsed 7.114285469055176
frame 5, elapsed 4.157355785369873
frame 10, elapsed 4.152865409851074
frame 15, elapsed 4.113606214523315
frame 20, elapsed 4.017664432525635
frame 25, elapsed 3.9587531089782715
frame 30, elapsed 3.9892005920410156
frame 35, elapsed 3.964831829071045
frame 40, elapsed 4.037587642669678
frame 45, elapsed 4.005861520767212
frame 50, elapsed 4.009392499923706
frame 55, elapsed 3.952181100845337
frame 60, elapsed 4.046564340591431
frame 65, elapsed 4.090888023376465
frame 70, elapsed 4.132278203964233
frame 75, elapsed 4.078840494155884
frame 80, elapsed 3.9970247745513916
frame 85, elapsed 3.982877492904663
frame 90, elapsed 4.049645900726318
frame 95, elapsed 4.020020008087158
frame 100, elapsed 3.9958608150482178
frame 105, elapsed 4.027854919433594
frame 110, elapsed 3.990032911300659
frame 115, elapsed 4.025416374206543
frame 120, elapsed 4.007601261138916
frame 125, elapsed 4.134798765182495
frame 130, elapsed 4.114675045013428
frame 135, elapsed 

frame 1110, elapsed 4.211045026779175
frame 1115, elapsed 4.209301471710205
frame 1120, elapsed 4.139425992965698
frame 1125, elapsed 4.0889387130737305
frame 1130, elapsed 4.087928295135498
frame 1135, elapsed 4.521927833557129
frame 1140, elapsed 4.246738433837891
frame 1145, elapsed 4.32978081703186
frame 1150, elapsed 4.203853130340576
frame 1155, elapsed 4.187920331954956
frame 1160, elapsed 4.120372295379639
frame 1165, elapsed 4.359119653701782
frame 1170, elapsed 8.142198085784912
frame 1175, elapsed 8.792668581008911
frame 1180, elapsed 7.7238500118255615
frame 1185, elapsed 4.861431360244751
frame 1190, elapsed 4.620843887329102
frame 1195, elapsed 4.213058710098267
frame 1200, elapsed 4.10561466217041
frame 1205, elapsed 4.098674535751343
frame 1210, elapsed 4.241865396499634
frame 1215, elapsed 4.236670255661011
frame 1220, elapsed 4.230310678482056
frame 1225, elapsed 4.0770792961120605
frame 1230, elapsed 4.1069495677948
frame 1235, elapsed 4.058038234710693
frame 1240, e

frame 2190, elapsed 4.39301609992981
frame 2195, elapsed 4.297546863555908
frame 2200, elapsed 4.2334229946136475
frame 2205, elapsed 4.176813364028931
frame 2210, elapsed 4.127718687057495
frame 2215, elapsed 4.101869344711304
frame 2220, elapsed 4.140906810760498
frame 2225, elapsed 4.298413991928101
frame 2230, elapsed 4.384717226028442
frame 2235, elapsed 4.236176013946533
frame 2240, elapsed 4.071791648864746
frame 2245, elapsed 4.173967123031616
frame 2250, elapsed 4.090896844863892
frame 2255, elapsed 4.400936603546143
frame 2260, elapsed 4.322774171829224
frame 2265, elapsed 4.201176404953003
frame 2270, elapsed 4.2031450271606445
frame 2275, elapsed 4.093860864639282
frame 2280, elapsed 4.214547872543335
frame 2285, elapsed 4.126896858215332
frame 2290, elapsed 4.20850944519043
frame 2295, elapsed 4.104019403457642
frame 2300, elapsed 4.057426929473877
frame 2305, elapsed 4.013312339782715
frame 2310, elapsed 4.021737098693848
frame 2315, elapsed 4.044513940811157
frame 2320, 

In [3]:
total_avg = []
for key, values in PER_GTID.items():
    print('actor ', key)
    print('\tuarms:', np.mean(values['uarms']))
    print('\tlarms:', np.mean(values['larms']))
    print('\tulegs:', np.mean(values['ulegs']))
    print('\tllegs:', np.mean(values['llegs']))
    avg = np.mean([
        np.mean(values['uarms']),
        np.mean(values['larms']),
        np.mean(values['ulegs']),
        np.mean(values['llegs'])
    ])
    total_avg.append(avg)
    print('\tavg:  ', avg)
print('\navg*:  ', np.mean(total_avg))

actor  0
	uarms: 0.7510245901639344
	larms: 0.6875
	ulegs: 0.9743852459016393
	llegs: 0.867827868852459
	avg:   0.8201844262295082
actor  1
	uarms: 0.6997950819672131
	larms: 0.7008196721311475
	ulegs: 0.9323770491803278
	llegs: 0.9293032786885246
	avg:   0.8155737704918034

avg*:   0.8178790983606558


## Like in the Master thesis
### alpha=0.5
* actor  0
	* uarms: 0.9764344262295082
	* larms: 0.9641393442622951
	* ulegs: 0.9897540983606558
	* llegs: 0.985655737704918
	* avg:   0.9789959016393442
* actor  1
	* uarms: 0.9682377049180327
	* larms: 0.9334016393442623
	* ulegs: 0.9877049180327869
	* llegs: 0.9805327868852459
	* avg:   0.967469262295082
* avg*:   0.9732325819672132

### alpha=0.2
* actor  0
	* uarms: 0.7510245901639344
	* larms: 0.6875
	* ulegs: 0.9743852459016393
	* llegs: 0.867827868852459
	* avg:   0.8201844262295082
* actor  1
	* uarms: 0.6997950819672131
	* larms: 0.7008196721311475
	* ulegs: 0.9323770491803278
	* llegs: 0.9293032786885246
	* avg:   0.8155737704918034
* avg*:   0.8178790983606558

## Like in Master thesis without Unaries
### alpha=0.5
* actor  0
	* uarms: 0.9774590163934426
	* larms: 0.9651639344262295
	* ulegs: 0.9897540983606558
	* llegs: 0.9866803278688525
	* avg:   0.979764344262295
* actor  1
	* uarms: 0.9692622950819673
	* larms: 0.9323770491803278
	* ulegs: 0.9877049180327869
	* llegs: 0.9805327868852459
	* avg:   0.967469262295082
* avg*:   0.9736168032786885

### alpha=0.2
* actor  0
	* uarms: 0.7510245901639344
	* larms: 0.6875
	* ulegs: 0.9733606557377049
	* llegs: 0.8668032786885246
	* avg:   0.819672131147541
* actor  1
	* uarms: 0.6987704918032787
	* larms: 0.6997950819672131
	* ulegs: 0.9313524590163934
	* llegs: 0.9282786885245902
	* avg:   0.8145491803278688
* avg*:   0.8171106557377049